## 一、导入相关的库

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.utils.data as data
import os
import random
from PIL import Image
from torchvision import models

# 设置随机种子
seed = 2020
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)            # 为CPU设置随机种子
torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)   # 为所有GPU设置随机种子
os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现。

## 二、获取文件路径和类别标签

In [ ]:
TRAIN_ROSES = '../input/demofortianchi/demo/data/train/roses/'
TRAIN_SUNFLOWERS = '../input/demofortianchi/demo/data/train/sunflowers/'
VAL_ROSES = '../input/demofortianchi/demo/data/val/roses/'
VAL_SUNFLOWERS = '../input/demofortianchi/demo/data/val/sunflowers/'

# 令roses类别为0，sunflower类别为1

train_roses = [TRAIN_ROSES + p for p in os.listdir(TRAIN_ROSES)]
train_sunflowers = [TRAIN_SUNFLOWERS + p for p in os.listdir(TRAIN_SUNFLOWERS)]
val_roses = [VAL_ROSES + p for p in os.listdir(VAL_ROSES)]
val_sunflowers = [VAL_SUNFLOWERS + p for p in os.listdir(VAL_SUNFLOWERS)]

train_paths = train_roses + train_sunflowers
val_paths = val_roses + val_sunflowers
train_labels = [0 for _ in range(len(train_roses))] + [1 for _ in range(len(train_sunflowers))]
val_labels = [0 for _ in range(len(val_roses))] + [1 for _ in range(len(val_sunflowers))]

# 需要将训练集进行打散，这样子训练的效果的会更好
train_paths = np.array(train_paths)
train_labels = np.array(train_labels)
p = np.random.permutation(len(train_labels))
train_paths = train_paths[p]
train_labels = train_labels[p]

print(train_paths)
print(val_paths)
print(train_labels)
print(val_labels)

1. ## 三、制作data loader类

In [ ]:
# data loader
class ImageDataset(data.Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.paths[index]
        label = self.labels[index]
        image = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)
        return image, int(label)

    def __len__(self):
        return len(self.paths)

In [ ]:
# 定义数据管道，
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
valid_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset = ImageDataset(train_paths, train_labels, train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)
validset = ImageDataset(val_paths, val_labels, valid_transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=False, num_workers=4)

## 四、搭建模型

In [ ]:
# 根据环境自动选择是否使用GPU
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
# 使用Alexnet网络进行图像分类
alexnet = models.alexnet(pretrained=True)
# 将alexnet最后一层全连接输出改为两个神经元，因为是二分类
alexnet.classifier[6].out_features = 2
alexnet.to(device)
# 打印alexnet网络结构
print(alexnet)

## 五、配置训练参数

In [ ]:
# 使用交叉熵损失进行训练
criterion = nn.CrossEntropyLoss()
# 使用随机梯度下降法进行优化模型，并且设置学习率为0.001
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)
# 训练10个epoch
EPOCH = 10

## 六、训练

In [ ]:
for epoch in range(EPOCH):
    running_loss = 0
    train_correct = 0
    train_total = 0
    for i, data in enumerate(trainloader):
        images, labels = data[0].to(device), data[1].to(device, dtype=torch.int64)
        optimizer.zero_grad()
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2 == 1:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/2))
            running_loss = 0.0
    train_accuracy = 100 * train_correct / train_total
    print('train dataset accuracy %.4f' % train_accuracy)

## 七、验证

In [ ]:
test_correct = 0
test_total = 0
res = []
with torch.no_grad():
    for data in validloader:
        images, labels = data[0].to(device), data[1].to(device, dtype=torch.int64)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        for p in predicted:
            res.append(int(p))

print("Val Accuracy: ", np.sum(np.array(res)==np.array(val_labels)) / len(val_labels))

## 八、保存模型

In [ ]:
PATH = 'model.pth'
torch.save(alexnet.state_dict(), PATH)